#  Machine Translation Challenge

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
callbacks_list = History()

Using TensorFlow backend.


In [2]:
#!pip install tensorflow==2.1.0
#!pip install keras==2.3.1

In [3]:
print('Kersa: ',keras.__version__,'Tensorflow: ', tf.__version__)

Kersa:  2.3.1 Tensorflow:  2.1.0


In [4]:
#Loading JW300_Data, Dropping ID column, removing all NaN
JW300_data_load = pd.read_csv("yoruba_data.csv")
pd.set_option('display.max_colwidth', -1)
JW300_data = JW300_data_load.drop(JW300_data_load.columns[0],axis = 1).dropna()

C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
JW300_data.sample(5)

,Yoruba,English
307984,Judson pa dà sẹ́nu iṣẹ́ ìtumọ̀ tó ń ṣe tẹ́lẹ̀ láìka gbogbo ìbànújẹ́ tó dé bá a yìí sí .,"Judson , though heartbroken , resumed his work ."
297881,4 / 1,3 / 15
38023,Èmi ni Jèhófà Ọlọ́run yín . ”,I am Jehovah your God . ”
472332,"Àwọn Júù kan tiẹ̀ ń bú Jésù , wọ́n ń pè é ní ará Samáríà .",Many Judeans viewed the people of Galilee as inferior .
47590,"Àmọ́ , ìṣòro kan wà o .","There was a problem , however ."


In [6]:
#Loading training data provided and dropping NaN
language_train_dataset = pd.read_csv("Language_Train.csv")
language_train_dataset = language_train_dataset.drop(language_train_dataset.columns[0],axis = 1).dropna()
pd.set_option('display.max_colwidth', -1)

C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [7]:
language_train_dataset.sample(5)

,Yoruba,English
7438,"Ni báyìí o tí di aàyò ọkàn mi ju ti tẹ́lẹ̀ lọ, ju bí mo ti lérò lọ́kàn lọ pàápàá.","At this moment he has become irreplaceable in my heart than before, I never expected it."
9263,"Àdàbà ń pògèdè, ó rò pé ẹyẹlé ò gbọ́; ẹyẹlé gbọ́, títiiri ló tiiri.","The dove recites incantations, thinking that the pigeon cannot hear; the pigeon hears; it is only pretending to sleep."
5453,"Ọgbọ́n àgbọ́njù ní ń sọ ẹni diwin; bí oògún bá pọ̀ lápọ̀jù a sọni di wèrè; bóbìnrín bá gbọ́n àgbọ́njù, péńpé laṣọ ọkọọ ẹ̀ ń mọ.",Excessive cleverness turns one into a phantom; if there is too much magical charm it turns the owner into an imbecile; if a woman is too cunning her husband's clothes wind up ill-fitting.
2920,"Ní April 20, 1933, nínú ètò kan tí wọ́n ṣe lórí rédíò láti ṣayẹyẹ ọjọ́ ìbí Hitler, àlùfáà ṣọ́ọ̀ṣì Lutheran kan tó ń jẹ́ Otto sọ pé: “Ṣọ́ọ̀ṣì Lutheran ti ìpínlẹ̀ Saxony ti ronú gan-an nípa ìjọba tuntun tó gorí àlééfà yìí, a sì ti pinnu pé a máa fọwọ́ sowọ́ pọ̀ pẹ̀lú ìjọba tuntun yìí láti mú kí orílẹ̀-èdè wa di alágbára, kí ẹ̀sìn Kristẹni sì gbèrú.","On April 20, 1933, in a radio broadcast honoring Hitler’s birthday, Lutheran minister Otto said: “The German Lutheran Church of the State of Saxony has consciously come to terms with the new situation and will attempt in closest cooperation with the political leaders of our people once again to make available to the entire nation the strength of the ancient gospel of Jesus Christ."
8322,--- Ìdíje ò̩dó̩ nílè̩ Adúláwò̩: Kwara káwọn 26 lọ̀pàgó̩ fúngbàáradì.,--- Africa Youth Games: 26 Kwara athletes invited to camp


In [8]:
data_frames = [language_train_dataset,JW300_data]
dataset1 =pd.concat(data_frames).drop_duplicates(keep=False)

In [9]:
dataset2 = dataset1[dataset1['English'].str.contains("www")]
dataset2

,Yoruba,English
18875,Gbogbo wọn àyàfi Leghorn Funfun : © Barry Koffler / www.feathersite.com,All except White Leghorn : © Barry Koffler / www.feathersite.com
20347,"X ray : Lọ́lá àṣẹ Ìgbìmọ̀ Tó Ń Ṣèwádìí Nípa Àrùn Oríkèé - Ara - Ríro , Ilẹ̀ Gẹ̀ẹ́sì ( www.arc.org.uk )","X ray : Used by kind permission of the Arthritis Research Campaign , United Kingdom ( www.arc.org.uk )"
23152,Ẹ̀YÌN ÌWÉ : UNITED NATIONS / RAY WITLIN,"COVER : Map used with permission , © RMC , www.randmcnally.com"
23891,© Randolph Langenbach / UNESCO ( www.conservationtech.com ),© Randolph Langenbach / UNESCO ( www.conservationtech.com )
27206,Fọ́tò tó wà lókè : http : /⁠ /⁠ www.constabulary.com,Photograph above : http : / / www.constabulary.com
...,...,...
466913,"Àwa Ẹlẹ́rìí Jèhófà la ṣe é , o tún lè rí i lórí ìkànnì wa www.jw.org / yo",published by Jehovah’s Witnesses and available online at www.jw.org .
467232,"Àwa Ẹlẹ́rìí Jèhófà la ṣe é , o tún lè wà á jáde lọ́fẹ̀ẹ́ lórí www.jw.org /⁠ yo",published by Jehovah’s Witnesses and available for free download at www.jw.org .
467263,"Àwa Ẹlẹ́rìí Jèhófà la ṣe é jáde , o sì lè wà á jáde lọ́fẹ̀ẹ́ lórí ìkànnì www.jw.org / yo .",published by Jehovah’s Witnesses and available for free download at www.jw.org .
467321,Ó tún wà lórí www.jw.org / yo,Also available at www.jw.org


In [10]:
dataset = pd.concat([dataset1, dataset2]).drop_duplicates(keep=False).convert_dtypes()

In [11]:
dataset

,Yoruba,English
0,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú.,A Disaster Relief Committee was formed to organize the long-term relief efforts.
1,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákùnrin Solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti Anna ìyàwó rẹ̀ ń bọ̀ láti ìrìn-àjò tí wọ́n lọ lórílẹ̀-èdè míì.","Brother Solovyev was arrested on the evening of May 22, 2018, at a railway station, as he was arriving home from a trip abroad with his wife, Anna."
2,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo̩lé, Senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù, wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú Colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ H ló̩jó̩bò̩.","With Egypt, Morocco and Tunisia out of the World Cup, Senegal, the only African nation left, will take on Columbia in their last group H game on Thursday."
4,"Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney General of the Federation), Justice Abubakar Malami ti pàsẹ fún ilé-isẹ́ ọ̀tẹlẹ̀múyẹ́ (Department of State Services, DSS) láti jáwọ́ lórí ẹ̀sun Omoyẹle Sowore ní kíákíá.","The Attorney General of the Federation, Justice Abubakar Malami has ordered the Department of State Services (DSS) to hands off the trial of Omoyele Sowore."
...,...,...
474963,"Lónìí , àwọn akéde tó lé ní ẹgbẹ̀rún lọ́nà àádọ́ta [ 50,000 ] ló ń wàásù ìhìn rere náà lórílẹ̀ - èdè Pọ́túgà àti láwọn erékùṣù míì tí wọ́n ti ń sọ èdè Potogí irú bí Azores àti Madeira .","Today , over 50,000 Kingdom publishers preach the good news of God’s Kingdom in Portugal and on several islands where Portuguese is spoken , including the Azores and Madeira ."
474964,Ó dùn mọ́ni pé lára àwọn akéde tó wà ní Pọ́túgà lónìí ló jẹ́ ìran kẹta lára àwọn tó gbọ́ àsọyé mánigbàgbé tí Arákùnrin Rutherford sọ lọ́dún 1925 .,Among these publishers today are third - generation descendants of some of those who attended Brother Rutherford’s historic lecture in 1925 .
474965,"A dúpẹ́ lọ́wọ́ Jèhófà , a sì dúpẹ́ lọ́wọ́ àwọn arákùnrin àtàwọn arábìnrin tí wọ́n fìgboyà wàásù ìhìn rere náà .",We give thanks to Jehovah and to those early faithful brothers and sisters who courageously took the lead in spearheading the work as ‘ public servants of Christ Jesus to the nations . ’ ​ — Rom .
474966,"Ṣe ni wọ́n fi hàn pé àwọn jẹ́ “ ìránṣẹ́ gbogbo ènìyàn fún Kristi Jésù sí àwọn orílẹ̀ - èdè . ” ​ — Róòmù 15 : 15 , 16 . ​ — Látinú Àpamọ́ Wa ní Pọ́túgà .","15 : 15 , 16 . ​ — From our archives in Portugal ."


In [12]:
#Fixing maximum scentence length as 20
dataset =dataset[dataset['Yoruba'].apply(lambda x: len(x.split(' ')) < 50)]

In [13]:
dataset

,Yoruba,English
0,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú.,A Disaster Relief Committee was formed to organize the long-term relief efforts.
1,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákùnrin Solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti Anna ìyàwó rẹ̀ ń bọ̀ láti ìrìn-àjò tí wọ́n lọ lórílẹ̀-èdè míì.","Brother Solovyev was arrested on the evening of May 22, 2018, at a railway station, as he was arriving home from a trip abroad with his wife, Anna."
2,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo̩lé, Senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù, wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú Colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ H ló̩jó̩bò̩.","With Egypt, Morocco and Tunisia out of the World Cup, Senegal, the only African nation left, will take on Columbia in their last group H game on Thursday."
4,"Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney General of the Federation), Justice Abubakar Malami ti pàsẹ fún ilé-isẹ́ ọ̀tẹlẹ̀múyẹ́ (Department of State Services, DSS) láti jáwọ́ lórí ẹ̀sun Omoyẹle Sowore ní kíákíá.","The Attorney General of the Federation, Justice Abubakar Malami has ordered the Department of State Services (DSS) to hands off the trial of Omoyele Sowore."
...,...,...
474963,"Lónìí , àwọn akéde tó lé ní ẹgbẹ̀rún lọ́nà àádọ́ta [ 50,000 ] ló ń wàásù ìhìn rere náà lórílẹ̀ - èdè Pọ́túgà àti láwọn erékùṣù míì tí wọ́n ti ń sọ èdè Potogí irú bí Azores àti Madeira .","Today , over 50,000 Kingdom publishers preach the good news of God’s Kingdom in Portugal and on several islands where Portuguese is spoken , including the Azores and Madeira ."
474964,Ó dùn mọ́ni pé lára àwọn akéde tó wà ní Pọ́túgà lónìí ló jẹ́ ìran kẹta lára àwọn tó gbọ́ àsọyé mánigbàgbé tí Arákùnrin Rutherford sọ lọ́dún 1925 .,Among these publishers today are third - generation descendants of some of those who attended Brother Rutherford’s historic lecture in 1925 .
474965,"A dúpẹ́ lọ́wọ́ Jèhófà , a sì dúpẹ́ lọ́wọ́ àwọn arákùnrin àtàwọn arábìnrin tí wọ́n fìgboyà wàásù ìhìn rere náà .",We give thanks to Jehovah and to those early faithful brothers and sisters who courageously took the lead in spearheading the work as ‘ public servants of Christ Jesus to the nations . ’ ​ — Rom .
474966,"Ṣe ni wọ́n fi hàn pé àwọn jẹ́ “ ìránṣẹ́ gbogbo ènìyàn fún Kristi Jésù sí àwọn orílẹ̀ - èdè . ” ​ — Róòmù 15 : 15 , 16 . ​ — Látinú Àpamọ́ Wa ní Pọ́túgà .","15 : 15 , 16 . ​ — From our archives in Portugal ."


# Selecting length of dataset

In [37]:
dataset = dataset.head(30000)

In [38]:
dataset = shuffle(dataset)

In [39]:
dataset

,Yoruba,English
7159,erékùṣù náà ní ẹwà àdánidá,START_ the island also has natural beauty to offer _END
13815,iṣẹ́ abẹ láìlo ẹ̀jẹ̀ ní gúúsù áfíríkà,START_ bloodless surgery in south africa _END
10631,ti ìdílé rẹ lẹ́yìn,START_ support your family _END
14682,nígbà tí wọ́n fẹ́ ti ọgbà ẹ̀wọ̀n kan tó wà ní ìlú cooma ní ọsirélíà pa àwọn èèyàn fárí gá,START_ when a prison in the small australian town of cooma was to be closed people protested _END
12246,graph àwòrán tó wà ní ojú ìwé,START_ graph picture on page _END
...,...,...
3829,ṣe ìwé ọkọ̀ náà ni orúkọ rẹ̀ kí o sì lo ìdánimọ̀ tìrẹ,START_ register the car in his name and use your biometrics _END
8596,creative commons bẹ̀rẹ̀ láti wá wọ̀rọ̀kọ̀ fi ṣe àdá látàrí ètò ẹ̀tọ́ àti àṣẹ ẹ̀dà lórí iṣẹ́ ọpọlọ àtinúdá àgbáńlá ayé tí kò bá ìgbà mu,START_ creative commons began in response to an outdated global copyright legal system _END
8359,láàárín ìlú ńlá mẹ́síkò gan an èèyàn lè ṣèbẹ̀wò sí tẹ́ńpìlì pàtàkì ti àwọn èèyàn aztec,START_ right in the heart of mexico city one can visit the main temple of the aztecs _END
18976,iye owó tí ìwà ipá ń náni ń ròkè sí i,START_ the soaring cost of crime _END


In [40]:
# convert Yoruba and English text to Lowercase 
dataset.Yoruba=dataset.Yoruba.apply(lambda x: x.lower())
dataset.English=dataset.English.apply(lambda x: x.lower())
# Remove quotes and hyphen from Yoruba and English text
dataset.Yoruba=dataset.Yoruba.apply(lambda x: re.sub("-", ' ', x))
dataset.English=dataset.English.apply(lambda x: re.sub("-", ' ', x))
dataset.Yoruba=dataset.Yoruba.apply(lambda x: re.sub("'", ' ', x))
dataset.English=dataset.English.apply(lambda x: re.sub("'", ' ', x))
dataset.Yoruba=dataset.Yoruba.apply(lambda x: re.sub("—", ' ', x))
dataset.English=dataset.English.apply(lambda x: re.sub("—", ' ', x))
# create a set of all special characters
special_characters= set(string.punctuation)
# Remove all the special characters
dataset.Yoruba = dataset.Yoruba.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
dataset.English = dataset.English.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
# Remove digits from Yoruba and English sentences
num_digits= str.maketrans('','', digits)
dataset.Yoruba=dataset.Yoruba.apply(lambda x: x.translate(num_digits))
dataset.English= dataset.English.apply(lambda x: x.translate(num_digits))
# Remove extra spaces
dataset.Yoruba=dataset.Yoruba.apply(lambda x: x.strip())
dataset.English=dataset.English.apply(lambda x: x.strip())
dataset.Yoruba=dataset.Yoruba.apply(lambda x: re.sub(" +", " ", x))
dataset.English=dataset.English.apply(lambda x: re.sub(" +", " ", x))


In [41]:
# Add start and end tokens to target sequences
dataset.English = dataset.English.apply(lambda x : 'START_ '+ x + ' _END')
dataset

,Yoruba,English
7159,erékùṣù náà ní ẹwà àdánidá,START_ start the island also has natural beauty to offer end _END
13815,iṣẹ́ abẹ láìlo ẹ̀jẹ̀ ní gúúsù áfíríkà,START_ start bloodless surgery in south africa end _END
10631,ti ìdílé rẹ lẹ́yìn,START_ start support your family end _END
14682,nígbà tí wọ́n fẹ́ ti ọgbà ẹ̀wọ̀n kan tó wà ní ìlú cooma ní ọsirélíà pa àwọn èèyàn fárí gá,START_ start when a prison in the small australian town of cooma was to be closed people protested end _END
12246,graph àwòrán tó wà ní ojú ìwé,START_ start graph picture on page end _END
...,...,...
3829,ṣe ìwé ọkọ̀ náà ni orúkọ rẹ̀ kí o sì lo ìdánimọ̀ tìrẹ,START_ start register the car in his name and use your biometrics end _END
8596,creative commons bẹ̀rẹ̀ láti wá wọ̀rọ̀kọ̀ fi ṣe àdá látàrí ètò ẹ̀tọ́ àti àṣẹ ẹ̀dà lórí iṣẹ́ ọpọlọ àtinúdá àgbáńlá ayé tí kò bá ìgbà mu,START_ start creative commons began in response to an outdated global copyright legal system end _END
8359,láàárín ìlú ńlá mẹ́síkò gan an èèyàn lè ṣèbẹ̀wò sí tẹ́ńpìlì pàtàkì ti àwọn èèyàn aztec,START_ start right in the heart of mexico city one can visit the main temple of the aztecs end _END
18976,iye owó tí ìwà ipá ń náni ń ròkè sí i,START_ start the soaring cost of crime end _END


In [42]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for Yoruba in dataset.Yoruba:
    for word in Yoruba.split():
        if word not in all_source_words:
            all_source_words.add(word)
# Vocabulary of Target 
all_target_words=set()
for English in dataset.English:
    for word in English.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

In [44]:
max(source_words, key=len)

'òjìlélẹ́gbọ̀kànléláàádọ́ta'

In [46]:
max(target_words, key=len)

'institutionalization'

In [47]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in dataset.Yoruba:
    source_length_list.append(len(l.split()))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)

target_length_list=[]
for l in dataset.English:
    target_length_list.append(len(l.split()))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 57
 Max length of the target sentence 75


In [48]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [73]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
#print(source_idx2word)
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])


In [50]:
# Train - Test Split
X, y = dataset.Yoruba, dataset.English
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((27000,), (3000,))

In [51]:
# Input tokens for encoder
num_encoder_tokens=len(source_words)
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [52]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = source_word2idx[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [53]:
train_samples = len(X_train)
val_samples = len(X_test)
#batch_size = 64
#epochs = 10
latent_dim=512

In [54]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [55]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [56]:
# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [57]:
model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['acc'])

# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [58]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 10
epochs = 40


In [59]:
my_model = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size,
                   callbacks=callbacks_list)

C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40
2700/2700 [==============================] - 658s 244ms/step - loss: 1.3521 - acc: 0.2479 - val_loss: 1.3592 - val_acc: 0.2801

Epoch 00001: val_acc improved from -inf to 0.28011, saving model to weights-improvement-01-0.28.h5
Epoch 2/40
2700/2700 [==============================] - 662s 245ms/step - loss: 1.2298 - acc: 0.2966 - val_loss: 1.3558 - val_acc: 0.3018

Epoch 00002: val_acc improved from 0.28011 to 0.30180, saving model to weights-improvement-02-0.30.h5
Epoch 3/40
2700/2700 [==============================] - 670s 248ms/step - loss: 1.1653 - acc: 0.3221 - val_loss: 1.3370 - val_acc: 0.3121

Epoch 00003: val_acc improved from 0.30180 to 0.31209, saving model to weights-improvement-03-0.31.h5
Epoch 4/40
2700/2700 [==============================] - 668s 247ms/step - loss: 1.1268 - acc: 0.3430 - val_loss: 1.3368 - val_acc: 0.3172

Epoch 00004: val_acc improved from 0.31209 to 0.31717, saving model to weights-improvement-04-0.32.h5
Epoch 5/40
2700/2700 [=================

In [60]:
model.save_weights("nmt_weights_30epochs.h5")

In [61]:
my_model.history

{'acc': [0.24791735,
  0.29655626,
  0.3220528,
  0.34304994,
  0.36160603,
  0.37669504,
  0.39021388,
  0.4008428,
  0.41170132,
  0.42286807,
  0.43484575,
  0.44892523,
  0.46414658,
  0.4744445,
  0.4828744,
  0.4926303,
  0.49959144,
  0.50585127,
  0.50926256,
  0.5177193,
  0.52404124,
  0.5326035,
  0.5397818,
  0.5565237,
  0.5645957,
  0.57221675,
  0.5778725,
  0.5873016,
  0.5978167,
  0.60930616,
  0.62000334,
  0.6303964,
  0.64147216,
  0.6520162,
  0.66249,
  0.6721859,
  0.6806489,
  0.6871115,
  0.6945918,
  0.6998649],
 'loss': [1.352126937618962,
  1.229768478296421,
  1.1653482453469877,
  1.1268014917329505,
  1.097837145725886,
  1.078417932567773,
  1.0665692814522318,
  1.050287369357215,
  1.0265886960316588,
  0.9940219768992177,
  0.9598113988615848,
  0.917471768811897,
  0.8766393972988482,
  0.8446034585877701,
  0.8176671956313981,
  0.7922090362398713,
  0.7722022276344123,
  0.7534791389659599,
  0.7417879841835411,
  0.7155594096801899,
  0.699229301

In [62]:
model.load_weights("nmt_weights_30epochs.h5")

In [63]:
from pandas import DataFrame
data = DataFrame(my_model.history)
data = data.round(decimals = 3)

In [64]:
data.to_csv('History.csv')

In [65]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [66]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']
# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
# Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
# Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
# Update states
        states_value = [h, c]
    return decoded_sentence

In [67]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [68]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print("Input Source sentence:", X_train[k:k+1].values[0])
print("Actual Target Translation:", y_train[k:k+1].values[0][6:-4])
print("Predicted Target Translation:", decoded_sentence[:-4])

Input Source sentence: mi ò sọ wí pé dípò kí mò ń sọ wí pé àti àrùn aids ikọ́ àwúgbẹ ibà ọ̀rẹ̀rẹ̀ ibà pọ́njúpọ́ntọ̀ oǹkà rẹ̀ tẹ̀síwájú
Actual Target Translation:  start i m not saying instead of i m saying as well as aids tb malaria typhoid the list goes on end 
Predicted Target Translation:  start i m saying i am victim i least as an addi


In [81]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=10
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: ṣé àwọn òbí ẹ ń ṣàròyé pé o kì í gba tẹlòmíràn rò pé anìkànjọpọ́n ni ẹ́
Actual Target Translation:  start do your parents complain that you are inconsiderate and self centered end 
Predicted Target Translation:  start take me back end 


# BLEU score

In [79]:
from nltk.translate.bleu_score import sentence_bleu
reference = [[ "àwọn", "míì", "tiẹ" ,"tún" ,"gbẹ́mìí" ,"mì" ,"torí", "pé" ,"wọn", "ò", "rí ","ìtọ́jú" ,"tó" ,"yẹ", "gbà"],
             ["some", "have", "also", "died", "because", "they" ,"could", "not" ,"obtain" ,"needed", "medical", "treatment"]]
candidate = ["some" ,"have", "also", "died" ,"because" ,"they", "did" ,"not", "have"]
score = sentence_bleu(reference, candidate)
print("BLEU score for the translation is :" ,score)

BLEU score for the translation is : 0.43742343691381713


In [87]:

reference = [[ "ṣé" ,"àwọn", "òbí" ,"ẹ", "ń", "ṣàròyé","pé", "o" ,"kì", "í" ,"gba", "tẹlòmíràn", "rò", "pé" ,"anìkànjọpón", "ni","ẹ́"],
             [ "start", "do" ,"your", "parents" ,"complain" ,"that" ,"you" ,"are", "inconsiderate", "and" ,"self", "centered" ,"end"]]
candidate = ["start" ,"take", "me", "back", "end" ]
score = sentence_bleu(reference, candidate)
print("BLEU score for the translation is :" ,score)

BLEU score for the translation is : 2.9251770022770788e-232


C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower 

In [91]:
from nltk.translate.bleu_score import sentence_bleu
reference = [["mi", "ò", "sọ" ,"wí" ,"pé" ,"dípò", "kí", "mò", "ń", "sọ" ,"wí", "pé" ,"àti" ,'àrùn' ,"aids", "ikọ" ,"àwúgbẹ" ,"ibà", "ọ̀rẹ̀rẹ̀" ,"ibà", "pọ́njúpọ́ntọ̀" ,"oǹkà", "rẹ ","tẹ̀síwájú" ],
             [ "i", "m", "not", "saying" ,"instead" ,"of", "i", "m", "saying" ,"as", "well", "as" ,"aids", "tb", "malaria", "typhoid", "the", "list", "goes", "on"]]
candidate = ["i", "m" ,"saying" ,"i", "am" ,"victim", "i", "least", "as" ,"an" ,"addi"]
score = sentence_bleu(reference, candidate)
print("BLEU score for the translation is :" ,score)

BLEU score for the translation is : 1.7084231998347353e-78


C:\Users\EngrS\anaconda3\envs\tensorflow\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
